# clone YOLOv5 repository before in content folder

In [ ]:
import torch
%pip install torch-utils
from IPython.display import Image  # to display images

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
%pip install roboflow
#init roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="roboflow-yolov5", api_key="the_api_key") #do not let this key appear anywhere

In [ ]:

# go to yolov5 folder
cd ./content/yolov5

In [ ]:
#install the dataset
project = rf.workspace("sammy-crveu").project("varroa-61n4t")
dataset = project.version(11).download("yolov5")

In [ ]:
#print the data.yaml specs
!cd {dataset.location} && type data.yaml

In [ ]:
# define number of classes based on YAML
# here varroa (1)
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#default configuration is sufficient
!cd ./models/ && type yolov5s.yaml && cd ..

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%time
#start the training, depending on the machine it can be between 30mins (mid tier tpu) and 15hours (mid tier GPU)
print(" dataset.location: ", dataset.location)
!python train.py --img 416 --batch 16 --epochs 30 --data {dataset.location}/data.yaml --cfg ./models/yolov5s.yaml --weights '' --name yolo_varroa --cache 

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source {dataset.location}/test/images

check result in /content/yolov5/runs/train/yolov5s_results/weights/best.pt